### Settings

In [ ]:
from IPython.core.display import display, HTML
import sys,cv2,gc
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas
from Utils.utils import *
from ipywidgets import interact
import deepdish as dd
from skimage import io, transform

%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
# set "context" (paper, notebook, talk, poster)
jtplot.style(theme='grade3',context='talk', fscale=2.5, spines=True, gridlines='',ticks=True, grid=True, figsize=(6, 4.5))
plotcolor = (0, 0.6, 1.0)

# Load augmentation stuff
from albumentations import (
    IAAFliplr,IAAFlipud, OneOf, RandomContrast, RandomBrightness, RandomGamma, RandomRotate90,
    IAASharpen, HueSaturationValue,IAAAdditiveGaussianNoise, MedianBlur, GaussNoise,
    IAAPiecewiseAffine, OneOf, Compose, Transpose,MotionBlur, Blur
)
def augment(p=1):
    return Compose([RandomRotate90(),IAAFliplr(),IAAFlipud(),Transpose(),
#         OneOf([GaussNoise(),], p=0.25),
#         OneOf([MotionBlur(p=.2),MedianBlur(blur_limit=3, p=.1),Blur(blur_limit=3, p=.1),], p=0.25),
        OneOf([RandomGamma(),RandomContrast(),RandomBrightness(),], p=0.5)
    ], p=p)

data_folder = 'D:/data/HPA/all/'
out_folder = 'D:/data/HPA/all/augmented/'
target_count = 1000 #set this lower than actual target count. will increase anyways
USE_ALL_CHANNELS = True
RESIZE = False
print('Done.')

### EDA

In [ ]:
%%time
#Read Labels
label_csv = pandas.read_csv(data_folder+'train.csv')
samplecount = label_csv['Id'].size
filenames = label_csv["Id"].values
labels = np.zeros([samplecount,28],dtype = np.bool)

for i, row in label_csv.iterrows():
    labelNr = list(map(int,row['Target'].split(' ')))
    labels[i,labelNr] = True #Convert labels to bool, where entry is true if class is present
    
frequencies = labels.sum(axis=0)
classes = np.argsort(frequencies)
frequencies.sort()
print(frequencies)

#shuffle labels and filenames
idx = np.arange(samplecount)
np.random.shuffle(idx)
filenames = filenames[idx].tolist()
labels = labels[idx]

In [ ]:
#for some reason we need to load theme twice...
# set "context" (paper, notebook, talk, poster)
jtplot.style(theme='grade3',context='talk', fscale=2.5, spines=True, gridlines='',ticks=True, grid=True, figsize=(6, 4.5))
plotcolor = (0, 0.6, 1.0)

plt.figure(figsize=(20,4))
plt.bar(range(28),frequencies,width = 0.5)
xtick_labels = list(map(str, classes))
plt.yscale('log')
plt.xlabel("Class Index")
plt.ylabel("Frequency")
_ = plt.xticks(range(28),xtick_labels)

### Sample and augment data
We sample in two steps:
- Get as many samples as possible or necessary from the present data
- Augment underrepresented classes

In [ ]:
#Setup
print("Collecting garbage...")
gc.collect()

X = []
selected_labels = []
class_representation = np.zeros(28)
print("Done.")

In [ ]:
#Step 1
for classNr in classes: #Iterate over classes starting with least represented ones
    print("######################## Sampling class: ", classNr, "Current class samples = ", class_representation[classNr], '######################## ')
    current_class_representation = class_representation[classNr]
    
    i = 0
    while(True): #Iterate over files
        #if enough samples collected for class, move on
        if current_class_representation >= target_count:
            break
        #iterating over changing stuff is nasty, this is our abort.
        if i >= len(filenames):
            break
            
        #check sample contains target class
        if labels[i][classNr]:
            #if sample present, add to dataset 
            fn = data_folder+'train/'+filenames[i]
            blue,green,red,yellow = cv2.imread(fn+'_blue.png',0),cv2.imread(fn+'_green.png',0),cv2.imread(fn+'_red.png',0),cv2.imread(fn+'_yellow.png',0)
            
            selected_labels.append(labels[i]) #store labels
            class_representation += labels[i] #update representation
            current_class_representation += 1
            
            # Store image, already handle resizing and channels
            if USE_ALL_CHANNELS: 
                all_img = np.asarray([green,red,blue,yellow]).transpose(1,2,0)
                if RESIZE:
                    all_img = transform.resize(all_img.squeeze(), (224, 224), preserve_range=True)
                cv2.imwrite(out_folder + filenames[i] +'_green.png',all_img[:,:,0].squeeze().astype(np.uint8))
                cv2.imwrite(out_folder + filenames[i] +'_red.png',all_img[:,:,1].squeeze().astype(np.uint8))
                cv2.imwrite(out_folder + filenames[i] +'_blue.png',all_img[:,:,2].squeeze().astype(np.uint8))
                cv2.imwrite(out_folder + filenames[i] +'_yellow.png',all_img[:,:,3].squeeze().astype(np.uint8))
                X.append(out_folder + filenames[i])
            else:
                if RESIZE:
                    green = transform.resize(green, (224, 224), preserve_range=True)
                cv2.imwrite(out_folder + filenames[i] +'_green.png',green.astype(np.uint8))
                X.append(out_folder + filenames[i])

            #Drop this sample from remaining data
            filenames = np.delete(filenames,i, axis=0)
            labels = np.delete(labels, i, axis=0)
            
            printProgressBar (current_class_representation, target_count, prefix = 'Sampling class...', suffix = '(' + str(current_class_representation) + '/' + str(target_count) + ')')
            
        i += 1
        
print_horizontal_divider()
print("Class representation after sampling: ", class_representation)
print("Done.")

In [ ]:
#we update the target number now, because oversampled the overrepresented classes for sure
#we aim for the middle ground between the overpresentation (the max) and the mean representation
# target_count = np.round((target_count + np.max(class_representation) - np.mean(class_representation)) / 2.0)
# print("New target count =", target_count)
# print(X[0].shape)
#store augmentations separately, we don't want to augment already augmented images
augmentedImages = [] 
augmentedLabels = []
# target_count = 100
print("Done.")

In [ ]:
#Step 2
for classNr in classes: #Iterate over classes starting with least represented ones
    current_class_representation = class_representation[classNr]
    i = 0
    iteration = 0
    while current_class_representation < target_count: # iterate as long as needed to get enough samples
        label = selected_labels[i]
        
        #check sample contains target class
        if label[classNr]:
            #augment image
            aug = augment()
            fn = X[i] + "_aug" + str(iteration)
            if USE_ALL_CHANNELS: 
                blue,green,red,yellow = cv2.imread(X[i]+'_blue.png',0),cv2.imread(X[i]+'_green.png',0),cv2.imread(X[i]+'_red.png',0),cv2.imread(X[i]+'_yellow.png',0)
                img_to_augment = np.asarray([green,red,blue,yellow]).transpose(1,2,0)
            else:
                img_to_augment = cv2.imread(X[i]+'_green.png',0)
                
            
            img = aug(image=img_to_augment.squeeze().astype(np.uint8))['image']

            #write image
            if USE_ALL_CHANNELS: 
                cv2.imwrite(fn +'_green.png',img[:,:,0].squeeze().astype(np.uint8))
                cv2.imwrite(fn +'_red.png',img[:,:,1].squeeze().astype(np.uint8))
                cv2.imwrite(fn +'_blue.png',img[:,:,2].squeeze().astype(np.uint8))
                cv2.imwrite(fn +'_yellow.png',img[:,:,3].squeeze().astype(np.uint8))
            else:
                cv2.imwrite(fn +'_green.png',img.squeeze().astype(np.uint8))
            
            #store filepath
            augmentedImages.append(fn)
            
            #add label
            augmentedLabels.append(label)
            
            #update class representation)
            class_representation += selected_labels[i] #update representation
            current_class_representation += 1
            
            if  i % 25 == 0 or (current_class_representation == target_count):
                printProgressBar (current_class_representation, target_count, prefix = 'Augmenting class ' + str(classNr) + '...', suffix = '(' + str(current_class_representation) + '/' + str(target_count) + ')')
        
        i += 1
        if len(selected_labels) == i:
            i = 0
            iteration += 1

#add augmented images
X.extend(augmentedImages)
selected_labels.extend(augmentedLabels)

print_horizontal_divider()
print("Final class representation: ", class_representation)
print("Total samples = ", len(X))
print("Done.")

### Store the created dataset

In [ ]:
%%time
# store data
data = {'filenames': np.asarray(X), 'labels': np.asarray(selected_labels)}
if USE_ALL_CHANNELS: 
    if RESIZE:
        dd.io.save(data_folder+'all_channel_augmented_small.h5', data,compression=('blosc', 8))
    else:
        dd.io.save(data_folder+'all_channel_augmented.h5', data,compression=('blosc', 8))
else:
    if RESIZE:
        dd.io.save(data_folder+'poi_augmented_small.h5', data,compression=('blosc', 8))
    else:
        dd.io.save(data_folder+'poi_augmented.h5', data,compression=('blosc', 8))
print("Done.")